In [ ]:
import numpy as np
# Load our library
import src.repex_topology_parser as rtp
# initialize our class providing an input processed.top file
test_module = rtp.topo2rest('tests/topology_files/test_topo/processed.top')

In [ ]:
def dihedraltype_array(dihedraltypes:list,exclude:list=[]):
   data = []
   dtype = np.dtype([('i','U10'),
                  ('j', 'U10'),  # 2nd string
                  ('k', 'U10'),  # 3rd string
                  ('l', 'U10'),  # 4th string
                  ('func', 'i4'),    # 5th integer
                  ('angle', 'f4'),    # 6th float
                  ('K', 'f4'),    # 7th float
                  ('mult', 'i4')])   # 8th integer])
   for string in dihedraltypes:
      line = string[:string.find(';')].split()
      if not len([i for i in line if i in exclude]):
         data.append((line[0], line[1], line[2], line[3], int(line[4]), float(line[5]), float(line[6]), int(line[7]))) 
   return np.array(data, dtype=dtype)

def dihedral_array(dihedraltypes:np.ndarray, dihedrals:list, atom_mapping:dict, verbose=False):
   data = []
   search_dihedrals = []
   dtype_dihedrals = np.dtype([('i','i4'),
                  ('j', 'i4'),  # 2nd string
                  ('k', 'i4'),  # 3rd string
                  ('l', 'i4'),  # 4th string
                  ('func', 'i4'),    # 5th integer
                  ('angle', 'f4'),    # 6th float
                  ('K', 'f4'),    # 7th float
                  ('mult', 'i4')])   # 8th integer])
   dtype = np.dtype([('i','U10'),
                  ('j', 'U10'),  # 2nd string
                  ('k', 'U10'),  # 3rd string
                  ('l', 'U10'),  # 4th string
                  ('func', 'U10'),    # 5th integer
                  ('angle', 'U10'),    # 6th float
                  ('K', 'U10'),    # 7th float
                  ('mult', 'U10')])   # 8th integer])
   for string in dihedrals:
      line = string[:string.find(';')].split()
      if len(line) == 5:
         search_dihedrals.append(tuple(map(int,line)))
      elif len(line) == 8:
         data.append((int(line[0]), int(line[1]), int(line[2]), int(line[3]), int(line[4]), float(line[5]), float(line[6]), int(line[7])))
      elif not line:
         continue
      else:
         raise RuntimeError(f'Line not processed: {line}')
   for (i, j, k, l, func) in search_dihedrals:
      if func == 4:
         mask = ((np.isin(dihedraltypes['i'], atom_mapping[i]) & \
               np.isin(dihedraltypes['j'], atom_mapping[j]) & \
               np.isin(dihedraltypes['k'], atom_mapping[k]) & \
               np.isin(dihedraltypes['l'], atom_mapping[l]) & \
               np.isin(dihedraltypes['func'], func)) | \
               (np.isin(dihedraltypes['i'], atom_mapping[l]) & \
               np.isin(dihedraltypes['j'], atom_mapping[k]) & \
               np.isin(dihedraltypes['k'], atom_mapping[j]) & \
               np.isin(dihedraltypes['l'], atom_mapping[i]) & \
               np.isin(dihedraltypes['func'], func))) & np.isin(dihedraltypes['func'], func)
         mask_Xl = (np.isin(dihedraltypes['i'], 'X') & \
               np.isin(dihedraltypes['j'], atom_mapping[j]) & \
               np.isin(dihedraltypes['k'], atom_mapping[k]) & \
               np.isin(dihedraltypes['l'], atom_mapping[l]) & \
               np.isin(dihedraltypes['func'], func)) 
         mask_Xll = (np.isin(dihedraltypes['i'], 'X') & \
               np.isin(dihedraltypes['j'], 'X') & \
               np.isin(dihedraltypes['k'], atom_mapping[k]) & \
               np.isin(dihedraltypes['l'], atom_mapping[l]) & \
               np.isin(dihedraltypes['func'], func))
         if verbose:
            if any(mask):
               print(f'found complete match for {(i,j,k,l,func)}')
               print(f'{" ".join([atom_mapping[atom] for atom in [i, j, k, l]])}')
               print(dihedraltypes[mask])
            elif any(mask_Xl):
               print(f'found match three for {(i, j, k, l,func)}')
               print(f'{" ".join([atom_mapping[atom] for atom in [i, j, k, l]])}')
               print(dihedraltypes[mask_Xl])
            elif any(mask_Xll):
               print(f'found match two for {(i, j, k, l,func)}')
               print(f'{" ".join([atom_mapping[atom] for atom in [i, j, k, l]])}')
               print(dihedraltypes[mask_Xll])
            else:
               raise LookupError(f'Dihedral not found: {[i, j, k, l, func].__str__()} {[atom_mapping[atom] for atom in [i, j, k, l]].__str__()}')
         elif not any(mask | mask_Xl | mask_Xll):
            raise LookupError(f'Dihedral not found: {[i, j, k, l, func].__str__()} {[atom_mapping[atom] for atom in [i, j, k, l]].__str__()}')
         
         if any(mask):
            for entries in dihedraltypes[mask]:
               data.append((i, j, k, l, func, entries[5], entries[6], entries[7])) 
         elif any(mask_Xl):
            for entries in dihedraltypes[mask_Xl]:
               data.append((i, j, k, l, func, entries[5], entries[6], entries[7])) 
         else:
            for entries in dihedraltypes[mask_Xll]:
               data.append((i, j, k, l, func, entries[5], entries[6], entries[7]))
            
      if func == 9:
         mask = ((np.isin(dihedraltypes['i'], atom_mapping[i]) & \
               np.isin(dihedraltypes['j'], atom_mapping[j]) & \
               np.isin(dihedraltypes['k'], atom_mapping[k]) & \
               np.isin(dihedraltypes['l'], atom_mapping[l]) & \
               np.isin(dihedraltypes['func'], func)) | \
               (np.isin(dihedraltypes['i'], atom_mapping[l]) & \
               np.isin(dihedraltypes['j'], atom_mapping[k]) & \
               np.isin(dihedraltypes['k'], atom_mapping[j]) & \
               np.isin(dihedraltypes['l'], atom_mapping[i]) & \
               np.isin(dihedraltypes['func'], func))) & np.isin(dihedraltypes['func'], func)
         mask_Xlr = ((np.isin(dihedraltypes['i'], 'X') & \
               np.isin(dihedraltypes['j'], atom_mapping[j]) & \
               np.isin(dihedraltypes['k'], atom_mapping[k]) & \
               np.isin(dihedraltypes['l'], 'X') & \
               np.isin(dihedraltypes['func'], func)) | \
               (np.isin(dihedraltypes['i'], 'X') & \
               np.isin(dihedraltypes['j'], atom_mapping[k]) & \
               np.isin(dihedraltypes['k'], atom_mapping[j]) & \
               np.isin(dihedraltypes['l'], 'X') & \
               np.isin(dihedraltypes['func'], func))) & np.isin(dihedraltypes['func'], func)
         if verbose:
            if any(mask):
               print(f'found complete match for {(i,j,k,l,func)}')
               print(f'{" ".join([atom_mapping[atom] for atom in [i, j, k, l]])}')
               print(dihedraltypes[mask])
            elif any(mask_Xlr):
               print(f'found match two for {[i, j, k, l]}')
               print(f'{" ".join([atom_mapping[i] for i in [i, j, k, l]])}')
               print(dihedraltypes[mask_Xlr])
            else:
               raise LookupError(f'Dihedral not found: {[i, j, k, l, func].__str__()} {[atom_mapping[atom] for atom in [i, j, k, l]].__str__()}')
         elif not any(mask | mask_Xlr):
            raise LookupError(f'Dihedral not found: {[i, j, k, l, func].__str__()} {[atom_mapping[atom] for atom in [i, j, k, l]].__str__()}')
         if any(mask): 
            for entries in dihedraltypes[mask]:
               data.append((i, j, k, l, func, entries[5], entries[6], entries[7])) 
         else:
            for entries in dihedraltypes[ mask_Xlr]:
               data.append((i, j, k, l, func, entries[5], entries[6], entries[7])) 
      
   
   structured_array = np.array(data, dtype=dtype_dihedrals)
   structured_array['func'] *= -1
   structured_array.sort(order=['l','k','j','mult','i'])
   structured_array.sort(order='func')
   structured_array['func'] *= -1
   
   return structured_array

def dihedrals_strings_list(dihedrals):
   stringsout = [f'{dihedral[0]:<5} {dihedral[1]:<5} {dihedral[2]:<5} {dihedral[3]:<5} {dihedral[4]:^9}' + \
                 f'{dihedral[5]:<10}{dihedral[5]:<10.5f}{dihedral[7]}\n' \
                 if len(dihedral) == 8 else f'{dihedral[0]:<5} {dihedral[1]:<5} {dihedral[2]:<5} {dihedral[3]:<5} {dihedral[4]:^9}' \
                 for dihedral in dihedrals ]
   return stringsout
dihedrals_strings_list(dihedral_array(dihedraltype_array([i for i in test_module._sections['dihedraltypes'] if len(i[:i.find(';')].split()) > 4]),\
   test_module._sections['moleculetype'][0]['dihedrals'],test_module.molecule_atoms[0]))

